# Notebook for collection of data + EDA
# James Yu

In [1]:
import pandas as pd
from random import sample
import sys
import json

## Looking at arXiv dataset from Kaggle

In [2]:
fp1 = '../data/arxivData.json'

In [3]:
# arXiv dataset from Kaggle https://www.kaggle.com/neelshah18/arxivdataset
df1 = pd.read_json(fp1)
df1.head()

,author,day,id,link,month,summary,tag,title,year
0,"[{'name': 'Ahmed Osman'}, {'name': 'Wojciech S...",1,1802.00209v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,We propose an architecture for VQA which utili...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Dual Recurrent Attention Units for Visual Ques...,2018
1,"[{'name': 'Ji Young Lee'}, {'name': 'Franck De...",12,1603.03827v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,Recent approaches based on artificial neural n...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Sequential Short-Text Classification with Recu...,2016
2,"[{'name': 'Iulian Vlad Serban'}, {'name': 'Tim...",2,1606.00776v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,We introduce the multiresolution recurrent neu...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Multiresolution Recurrent Neural Networks: An ...,2016
3,"[{'name': 'Sebastian Ruder'}, {'name': 'Joachi...",23,1705.08142v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,Multi-task learning is motivated by the observ...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Learning what to share between loosely related...,2017
4,"[{'name': 'Iulian V. Serban'}, {'name': 'Chinn...",7,1709.02349v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,We present MILABOT: a deep reinforcement learn...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",A Deep Reinforcement Learning Chatbot,2017


In [9]:
df1.value_counts('year')

year
2017    12017
2016     6265
2015     4467
2018     3412
2013     3318
2014     3219
2012     2555
2011     1522
2010      975
2009      689
2008      494
2007      325
2006      245
2000      224
2004      212
2005      204
2002      194
2003      188
2001      126
1994      122
1999       77
1998       68
1996       28
1995       28
1997       20
1993        6
dtype: int64

In [10]:
df1.value_counts('month')

month
3     4192
11    4164
6     3766
2     3752
12    3553
9     3405
10    3377
1     3162
7     3104
5     2966
4     2819
8     2740
dtype: int64

In [29]:
agg_titles_by_year = df1.groupby('year')['title'].apply(' '.join)
agg_titles_by_year.head()

year
1993    Dynamic Backtracking A Market-Oriented Program...
1994    Teleo-Reactive Programs for Agent Control Lear...
1995    Solving Multiclass Learning Problems via Error...
1996    The Design and Experimental Analysis of Algori...
1997    Machine Learning of User Profiles: Representat...
Name: title, dtype: object

In [28]:
agg_summ_by_year = df1.groupby('year')['summary'].apply(' '.join)
agg_summ_by_year.head()

year
1993    Because of their occasional need to return to ...
1994    A formalism is presented for computing and org...
1995    Multiclass learning problems involve finding a...
1996    Many applications -- from planning and schedul...
1997    As more information becomes available electron...
Name: summary, dtype: object

In [27]:
agg_all_by_year = df1.groupby('year')['title'].apply(' '.join) + ' ' + df1.groupby('year')['summary'].apply(' '.join)
agg_all_by_year.head()

year
1993    Dynamic Backtracking A Market-Oriented Program...
1994    Teleo-Reactive Programs for Agent Control Lear...
1995    Solving Multiclass Learning Problems via Error...
1996    The Design and Experimental Analysis of Algori...
1997    Machine Learning of User Profiles: Representat...
dtype: object

In [37]:
agg_all_by_year[1993][:500]

'Dynamic Backtracking A Market-Oriented Programming Environment and its Application to\n  Distributed Multicommodity Flow Problems An Empirical Analysis of Search in GSAT The Difficulties of Learning Logic Programs with Cut Software Agents: Completing Patterns and Constructing User Interfaces Decidable Reasoning in Terminological Knowledge Representation Systems Because of their occasional need to return to shallow points in a search\ntree, existing backtracking methods can sometimes erase meaningf'

In [38]:
agg_all_by_year[1993][:500].replace('\n', ' ')

'Dynamic Backtracking A Market-Oriented Programming Environment and its Application to   Distributed Multicommodity Flow Problems An Empirical Analysis of Search in GSAT The Difficulties of Learning Logic Programs with Cut Software Agents: Completing Patterns and Constructing User Interfaces Decidable Reasoning in Terminological Knowledge Representation Systems Because of their occasional need to return to shallow points in a search tree, existing backtracking methods can sometimes erase meaningf'

## Looking at DBLP dataset

In [4]:
fp2 = '../data/dblpv13.json'

In [5]:
# DBLP v13 dataset
# file size is 16.1 GB
# Can try to read in as smaller pieces, or write out the data into .txt files by year

#df2 = pd.read_json(fp2)
#df2.head()

In [6]:
#json.load(open(fp2, "r"))